In [1]:
import json
from pathlib import Path
corpus_name = 'conll2003'
model_name = 'bert'
variant = f'{corpus_name}_{model_name}'

# corpus_file = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'
corpus_file = '/Users/ahmed/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'


with open(corpus_file, 'r') as file:
    corpora = json.load(file)  # Use json.load() to read file, not json.loads()




In [2]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import plotly.express as px
from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities, auto_detect

from collections import Counter, defaultdict
import scipy.stats as stats

from collections import Counter
import pandas as pd


In [3]:
def calculate_general_stats(corpora_splits):
    stats = []
    all_tokens = []
    all_entity_tokens = []  # Collect tokens that are part of entities
    all_unique_entity_words = set()

    for split_name, split_data in corpora_splits.items():
        tokens = []
        entity_tokens = []  # Tokens tagged as entities
        unique_entity_words = set()

        # Extract tokens and entity tokens
        for sentence in split_data:
            tokens.extend(sentence['words'])
            for word, tag in zip(sentence['words'], sentence['tags']):
                if tag != "O":  # If not "O", it's part of an entity
                    entity_tokens.append(word)
                    unique_entity_words.add(word)

        total_tokens = len(tokens)
        unique_words = len(set(tokens))
        ttr = unique_words / total_tokens if total_tokens else 0
        ne_proportion = len(entity_tokens) / total_tokens if total_tokens else 0
        entity_type_proportion = len(unique_entity_words) / len(entity_tokens) if len(entity_tokens) else 0


        # Collect all tokens and entity tokens for overall calculation
        all_tokens.extend(tokens)
        all_entity_tokens.extend(entity_tokens)
        all_unique_entity_words.update(unique_entity_words)

        # Add split-level stats
        stats.append({
            'Split': split_name,
            'Total Tokens': total_tokens,
            'Unique Words': unique_words,
            'TTR': round(ttr, 4),
            'Entity Words': len(entity_tokens),
            'NE Proportion': round(ne_proportion, 4),
            'Entity Types': len(unique_entity_words),
            'ETTR': round(entity_type_proportion, 4),
        })

    # Overall dataset stats
    total_tokens = len(all_tokens)
    unique_words = len(set(all_tokens))
    ttr = unique_words / total_tokens if total_tokens else 0
    ne_proportion = len(all_entity_tokens) / total_tokens if total_tokens else 0
    entity_type_proportion = len(all_unique_entity_words) / len(all_entity_tokens) if len(all_entity_tokens) else 0

    stats.append({
        'Split': 'Total',
        'Total Tokens': total_tokens,
        'Unique Words': unique_words,
        'TTR': round(ttr, 4),
        'Entity Words': len(all_entity_tokens),
        'NE Proportion': round(ne_proportion, 4),
        'Entity Types': len(all_unique_entity_words),
        'ETTR': round(entity_type_proportion, 4),
    })

    # Convert to DataFrame
    df = pd.DataFrame(stats)
    return df



def calculate_tag_stats(corpora_splits, tag_set):
    stats = []
    tag_totals = Counter({tag: 0 for tag in tag_set})
    all_unique_tag_words = {tag: set() for tag in tag_set}

    # Process splits
    for split_name, split_data in corpora_splits.items():
        tag_counts = Counter({tag: 0 for tag in tag_set})
        unique_tag_words = {tag: set() for tag in tag_set}

        total_ne_tokens = 0  # Total NE tokens for proportions
        total_ne_unique_words = set()

        # Process each sentence
        for sentence in split_data:
            for word, tag in zip(sentence['words'], sentence['tags']):
                if tag in tag_counts:
                    tag_counts[tag] += 1
                    unique_tag_words[tag].add(word)
                    if tag != "O":  # Count only NE tokens for proportions
                        total_ne_tokens += 1
                        total_ne_unique_words.add(word)

        # Calculate stats for each tag
        for tag, count in tag_counts.items():
            tag_token_proportion = count / total_ne_tokens if total_ne_tokens else 0
            tag_type_proportion = len(unique_tag_words[tag]) / len(total_ne_unique_words) if total_ne_unique_words else 0
            ttr = len(unique_tag_words[tag]) / count if count else 0

            stats.append({
                'Split': split_name,
                'Tag': tag,
                'Tag Words': count,
                'Tag Types': len(unique_tag_words[tag]),
                'TTR': round(ttr, 4),
                'Words Proportion': round(tag_token_proportion, 4),
                'Type Proportion': round(tag_type_proportion, 4),
            })

        # Update totals for overall stats
        tag_totals.update(tag_counts)
        for tag in tag_set:
            all_unique_tag_words[tag].update(unique_tag_words[tag])

    # Overall dataset stats
    total_ne_tokens = sum(tag_totals[tag] for tag in tag_set if tag != "O")
    total_ne_unique_words = set(word for tag in tag_set if tag != "O" for word in all_unique_tag_words[tag])

    for tag, count in tag_totals.items():
        tag_token_proportion = count / total_ne_tokens if total_ne_tokens else 0
        tag_type_proportion = len(all_unique_tag_words[tag]) / len(total_ne_unique_words) if total_ne_unique_words else 0
        ttr = len(all_unique_tag_words[tag]) / count if count else 0

        # stats.append({
        #     'Split': 'Total',
        #     'Tag': tag,
        #     'Tag Tokens': count,
        #     'Tag Types': len(all_unique_tag_words[tag]),
        #     'TTR': round(ttr, 4),
        #     'Token Proportion': round(tag_token_proportion, 4),
        #     'Type Proportion': round(tag_type_proportion, 4),
        # })

    # Convert to DataFrame
    df = pd.DataFrame(stats)
    return df


# Combine stats for both ANERCorp and CoNLL
def calculate_combined_stats(corpora):
    anercorp_splits = corpora['ANERCorp_CamelLab']['splits']
    conll_splits_no_validation = {key: value for key, value in corpora['conll2003']['splits'].items() if key != 'validation'}

    # Calculate general stats
    anercorp_general = calculate_general_stats(anercorp_splits)
    conll_general = calculate_general_stats(conll_splits_no_validation)

    # Add dataset source
    anercorp_general['Dataset'] = 'ANERCorp_CamelLab'
    conll_general['Dataset'] = 'CoNLL-2003'

    # Combine general stats
    combined_general = pd.concat([anercorp_general, conll_general], ignore_index=True)

    # Calculate tag stats
    anercorp_tag_set = ['B-LOC', 'I-LOC', 'B-PERS', 'I-PERS', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
    anercorp_tags = calculate_tag_stats(anercorp_splits, anercorp_tag_set)
    conll_tag_set = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
    conll_tags = calculate_tag_stats(conll_splits_no_validation, conll_tag_set)

    # Add dataset source
    anercorp_tags['Dataset'] = 'ANERCorp_CamelLab'
    conll_tags['Dataset'] = 'CoNLL-2003'

    # Combine tag stats
    combined_tags = pd.concat([anercorp_tags, conll_tags], ignore_index=True)

    return combined_general, combined_tags

def calculate_oov_rates(corpora):
    results = {}

    for corpus_name, corpus_data in corpora.items():
        if corpus_name != 'ANERCorp_CamelLab-validation':
            # Ensure that both 'train' and 'test' splits are available
            if 'train' in corpus_data['splits'] and 'test' in corpus_data['splits']:
                # Extract unique words from training and testing sets
                train_words = set(
                    w for sentence in corpus_data['splits']['train'] for w in sentence['words']
                )
                test_words = set(
                    w for sentence in corpus_data['splits']['test'] for w in sentence['words']
                )

                # Find the OOV words
                oov_words = test_words - train_words

                # Calculate OOV rate
                oov_rate = len(oov_words) / len(test_words) if test_words else 0

                # Store results for this corpus
                results[corpus_name] = {
                    'OOV Words Count': len(oov_words),
                    'Total Unique Words in Test': len(test_words),
                    'OOV Rate': round(oov_rate, 4)
                }
            else:
                results[corpus_name] = "Missing 'train' or 'test' split data"

    return results


def calculate_oov_rates_per_tag(corpora):
    results = {}

    for corpus_name, corpus_data in corpora.items():
        if corpus_name != 'ANERCorp_CamelLab-validation':
            if 'train' in corpus_data['splits'] and 'test' in corpus_data['splits']:
                train_words = {}
                test_words = {}
                
                # Collect words and their associated tags from training set
                for sentence in corpus_data['splits']['train']:
                    for word, tag in zip(sentence['words'], sentence['tags']):
                        normalised_tag = tag.replace('PERS', 'PER')
                        if normalised_tag not in train_words:
                            train_words[normalised_tag] = set()
                        train_words[normalised_tag].add(word)
                
                # Collect words and their associated tags from test set
                for sentence in corpus_data['splits']['test']:
                    for word, tag in zip(sentence['words'], sentence['tags']):
                        normalised_tag = tag.replace('PERS', 'PER')
                        if normalised_tag not in test_words:
                            test_words[normalised_tag] = set()
                        test_words[normalised_tag].add(word)
                
                # Calculate OOV rate per tag
                oov_stats = {}
                for tag in test_words:
                    if tag != 'O':
                        oov_words = test_words[tag] - train_words.get(tag, set())
                        oov_rate = len(oov_words) / len(test_words[tag]) if test_words[tag] else 0
                        oov_stats[tag] = {
                            'OOV Words Count': len(oov_words),
                            'Total Unique Words in Test': len(test_words[tag]),
                            'OOV Rate': round(oov_rate, 4)
                        }
                
                results[corpus_name] = oov_stats
            else:
                results[corpus_name] = "Missing 'train' or 'test' split data"

    return results



def get_entity_distribution(corpora, split):
    data = []
    datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
    versions = ['IOB1', 'IOB2']

    for dataset in datasets_of_interest:
        for version in versions:
            y_true = [sen['tags'] for sen in corpora[dataset]['splits'][split]]
            if version == 'IOB1':
                true_entities = get_entities(y_true)
            else:
                scheme = auto_detect(y_true, False)
                entities = Entities(y_true, scheme, False)
                true_entities = [entity.to_tuple()[1:] for sen in entities.entities for entity in sen]
            raw_counts = Counter([entity[0].replace('PERS', 'PER') for entity in true_entities])

            total = sum(raw_counts.values())
            for entity_type, count in raw_counts.items():
                percentage = round((count / total * 100), 2) if total > 0 else 0
                data.append({
                    'Dataset': dataset,
                    'Scheme Version': version,
                    'Entity Type': entity_type,
                    'Raw Count': count,
                    'Proportion': percentage
                })

    df = pd.DataFrame(data)
    return df

def calculate_entity_lengths(entities, strict=False):
    # Initialize a dictionary to store lengths of each entity type
    entity_lengths = defaultdict(list)

    # Calculate the length of each entity and store it in the dictionary
    for entity_type, start, end in entities:
        
        if strict:
            length = end - start
        else:
            length = end - start + 1  # +1 because end index is inclusive
        
        entity_lengths[entity_type.replace('PERS', 'PER')].append(length)

    # Compute average length and other statistics for each entity type
    entity_stats = {}
    for entity_type, lengths in entity_lengths.items():
        average_length = round(np.mean(lengths), 4)
        std = round(np.std(lengths), 4)
        min_length = min(lengths)
        max_length = max(lengths)
        entity_stats[entity_type] = {
            'Average Length': average_length,
            'Min Length': min_length,
            'Max Length': max_length,
            'STD': std
        }

    return entity_stats



def process_datasets(corpora, split):
    results = []
    datasets_of_interest = ['ANERCorp_CamelLab', 'conll2003']
    for dataset_name, dataset in corpora.items():
        if dataset_name in datasets_of_interest:
            for version in ['IOB1', 'IOB2']:
                y_true = [sen['tags'] for sen in dataset['splits'][split]]
                strict = version == 'IOB2'
                if strict:
                    scheme = auto_detect(y_true, False)
                    true_entities = Entities(y_true, scheme, False)
                    entity_stats = calculate_entity_lengths([entity.to_tuple()[1:] for sen in true_entities.entities for entity in sen], True)
                else:
                    true_entities = get_entities(y_true)
                    entity_stats = calculate_entity_lengths(true_entities)
                    
                # Append results for DataFrame conversion
                for entity_type, stats in entity_stats.items():
                    stats.update({
						'Dataset': dataset_name,
						'Scheme': version,
						'Entity Type': entity_type
					})
                    results.append(stats)

    return pd.DataFrame(results)



import pandas as pd
import plotly.express as px

class EntityTagVisualizer:
    def __init__(self, df):
        self.df = df
        self.color_map = {
            "B-LOC": "darkgreen",
            "B-PERS": "deepskyblue",
            "B-PER": "deepskyblue",
            "B-ORG": "darkcyan",
            "B-MISC": "palevioletred",
            "I-LOC": "yellowgreen",
            "I-PERS": "lightblue",
            "I-PER": "lightblue",
            "I-ORG": "cyan",
            "I-MISC": "violet",
            "O": "saddlebrown",
        }
        self.data_map = {
            "ANERCorp_CamelLab": "ANERCorp"
        }
        self.label_map = {
            "B-PERS": "B-PER",
            "I-PERS": "I-PER"
        }
        self.split_map = {
            "train": "Train",
            "test": "Test"
        }

    def preprocess_data(self):
        # Apply mappings to standardize dataset names, tag labels, and split names
        self.df['Dataset'] = self.df['Dataset'].replace(self.data_map)
        self.df['Tag'] = self.df['Tag'].replace(self.label_map)
        self.df['Split'] = self.df['Split'].replace(self.split_map)

    def plot_faceted_bar_chart(self, metric, text, title):
        # Create a faceted bar chart
        fig = px.bar(
            self.df,
            x='Tag',
            y=metric,
            color='Tag',  # Color coding by tag type
            facet_col='Dataset',
            facet_row='Split',
            color_discrete_map=self.color_map,
            text=text,
            title=title,
            labels={"Words Proportion": "Proportion of Words", "Tag": "Entity Tag"}
        )

        # Update layout for better presentation
        fig.update_layout(
            template="plotly_white",
            height=600,
            width=1200,
            margin=dict(t=60, l=20, r=20, b=20)
        )

        # Display the plot
        fig.show()

import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots

class OverlapMatrix:
    def __init__(self, corpora, tags):
        self.corpora = corpora
        self.tags = tags

    def extract_O_overlap_matrix(self, data):
        tag_types = {tag: set() for tag in self.tags}
        for sentence in data:
            for word, tag in zip(sentence['words'], sentence['tags']):
                normalised_tag = tag.replace('PERS', 'PER')  # Normalizing tags like 'PERS'
                if normalised_tag in self.tags:
                    tag_types[normalised_tag].add(word)
                elif tag == 'O':
                    tag_types['O'].add(word)

        overlap_matrix = pd.DataFrame(index=self.tags, columns=self.tags, dtype=int).fillna(0)
        for tag1 in self.tags:
            for tag2 in self.tags:
                if tag1 != tag2:
                    overlap_matrix.loc[tag1, tag2] = len(tag_types[tag1].intersection(tag_types[tag2]))

        return overlap_matrix

    def visualize_matrices(self, data, split):
        fig = make_subplots(
            rows=1, cols=len(data),
            subplot_titles=[name for name in data.keys()],
            horizontal_spacing=0.1
        )

        col = 1
        for name, dataset in data.items():
            matrix = self.extract_O_overlap_matrix(self.corpora[dataset]['splits'][split])
            mask_lower = np.tril(np.ones_like(matrix, dtype=bool))
            lower_triangle = matrix.mask(~mask_lower)
            text_data = np.where(lower_triangle.isnull(), '', lower_triangle.fillna(0).astype(int).astype(str))

            fig.add_trace(
                go.Heatmap(
                    z=lower_triangle,
                    x=lower_triangle.columns,
                    y=lower_triangle.index,
                    colorscale="RdBu_r",
                    showscale=True,
                    text=text_data,
                    texttemplate="%{text}",
                    hoverinfo="text+z"
                ),
                row=1, col=col
            )
            col += 1
        split_title = ' for the Training Split' if split=='train' else ' for the Testing Split'
        fig.update_layout(
            title_text='Word Type Overlap Across Entity Tags' + split_title,
            template="plotly_white",
            height=600,
            width=1200,
            xaxis=dict(showgrid=False),  # Hide gridlines for x-axis
            xaxis2=dict(showgrid=False), # Hide gridlines for x-axis in second subplot
            yaxis=dict(showgrid=False),  # Hide gridlines for y-axis
            yaxis2=dict(showgrid=False), # Hide gridlines for y-axis in second subplot
        )
        fig.show()


class EntityTagTTRVisualizer:
    def __init__(self, df):
        self.df = df
        self.color_map = {
            "B-LOC": "darkgreen",
            "B-PERS": "deepskyblue",
            "B-PER": "deepskyblue",
            "B-ORG": "darkcyan",
            "B-MISC": "palevioletred",
            "I-LOC": "yellowgreen",
            "I-PERS": "lightblue",
            "I-PER": "lightblue",
            "I-ORG": "cyan",
            "I-MISC": "violet",
            "O": "saddlebrown",
        }
        self.data_map = {
            "ANERCorp_CamelLab": "ANERCorp"
        }
        self.label_map = {
            "B-PERS": "B-PER",
            "I-PERS": "I-PER"
        }
        self.split_map = {
            "train": "Train",
            "test": "Test"
        }

    def preprocess_data(self):
        """
        Preprocesses the DataFrame by standardizing dataset names, tag labels, and split names
        according to the provided mappings.
        """
        self.df['Dataset'] = self.df['Dataset'].replace(self.data_map)
        self.df['Tag'] = self.df['Tag'].replace(self.label_map)
        self.df['Split'] = self.df['Split'].replace(self.split_map)
        self.df['TWR'] = self.df['TTR'].round(2)
        

    def plot_faceted_bar_chart(self, metric, text, title):
        """
        Plots a faceted bar chart based on the specified metric, with facets for datasets and splits.

        Parameters:
            metric (str): The column in the DataFrame to be used for the y-axis.
            text (str): The column to be used as labels on the bars.
            title (str): The title of the chart.
        """
        # Create a faceted bar chart
        fig = px.bar(
            self.df,
            x='Tag',
            y=metric,
            color='Tag',  # Color coding by tag type
            facet_col='Dataset',
            facet_row='Split',
            color_discrete_map=self.color_map,
            text=text,
            title=title,
            labels={"Words Proportion": "Proportion of Words", "Tag": "Entity Tag"}
        )

        # Update layout for better presentation
        fig.update_layout(
            template="plotly_white",
            height=700,
            width=1200,
            margin=dict(t=60, l=20, r=20, b=20),
            title_x=0.5  # Center align the title
        )

        # Update text formatting
        fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

        # Display the plot
        fig.show()
        


In [4]:
combined_general_stats, combined_tag_stats = calculate_combined_stats(corpora)
anercrop = combined_tag_stats[combined_tag_stats['Dataset'] == 'ANERCorp_CamelLab'].copy()

In [5]:
combined_general_stats

,Split,Total Tokens,Unique Words,TTR,Entity Words,NE Proportion,Entity Types,ETTR,Dataset
0,train,125102,29252,0.2338,13181,0.1054,4069,0.3087,ANERCorp_CamelLab
1,test,25008,9075,0.3629,3375,0.1350,1603,0.4750,ANERCorp_CamelLab
2,Total,150110,32714,0.2179,16556,0.1103,5026,0.3036,ANERCorp_CamelLab
3,train,203621,23623,0.1160,34043,0.1672,8287,0.2434,CoNLL-2003
4,test,46435,9488,0.2043,8112,0.1747,3129,0.3857,CoNLL-2003
5,Total,250056,27316,0.1092,42155,0.1686,9819,0.2329,CoNLL-2003


In [14]:
oov_stats = calculate_oov_rates(corpora)
pd.DataFrame(oov_stats)


,ANERCorp_CamelLab,conll2003
OOV Words Count,3462.0000,3693.0000
Total Unique Words in Test,9075.0000,9488.0000
OOV Rate,0.3815,0.3892


# Token Level

In [15]:

# Example usage
# df is your DataFrame loaded with the necessary 'Dataset', 'Tag', 'Words Proportion', and 'Tag Words' columns
visualizer = EntityTagVisualizer(df=anercrop)
visualizer.preprocess_data()
visualizer.plot_faceted_bar_chart('Words Proportion', 'Tag Words', "Entity Tag Words Distribution Across Training and Testing Splits",)


In [16]:
# Step 1: Calculate the total tag types for each Dataset and Split
total_tag_types = anercrop.groupby(['Dataset', 'Split'])['Tag Types'].sum().reset_index()
total_tag_types.rename(columns={'Tag Types': 'Total Tag Types'}, inplace=True)

# Step 2: Merge the total back to the original DataFrame to calculate proportions
df = anercrop.merge(total_tag_types, on=['Dataset', 'Split'])

# Step 3: Calculate the proportion of tag types within each group
df['Tag Type Proportion'] = df['Tag Types'] / df['Total Tag Types']

df
visualizer = EntityTagVisualizer(df=df)
visualizer.preprocess_data()

visualizer.plot_faceted_bar_chart('Tag Type Proportion', 'Tag Types', "Entity Tag Word Types Distribution Across Training and Testing Splits",)

In [17]:
total_tag_types

,Dataset,Split,Total Tag Types
0,ANERCorp,Test,1822
1,ANERCorp,Train,4735


In [18]:

class OverlapMatrixOneScale:
    def __init__(self, corpora, tags):
        self.corpora = corpora
        self.tags = tags

    def extract_O_overlap_matrix(self, data):
        tag_types = {tag: set() for tag in self.tags}
        for sentence in data:
            for word, tag in zip(sentence['words'], sentence['tags']):
                normalised_tag = tag.replace('PERS', 'PER')
                if normalised_tag in self.tags:
                    tag_types[normalised_tag].add(word)
                elif tag == 'O':
                    tag_types['O'].add(word)

        overlap_matrix = pd.DataFrame(index=self.tags, columns=self.tags, dtype=int).fillna(0)
        for tag1 in self.tags:
            for tag2 in self.tags:
                if tag1 != tag2:
                    overlap_matrix.loc[tag1, tag2] = len(tag_types[tag1].intersection(tag_types[tag2]))

        return overlap_matrix

    def visualize_matrices(self, data, split):
        fig = make_subplots(
            rows=1, cols=len(data),
            subplot_titles=[name for name in data.keys()],
            horizontal_spacing=0.1
        )

        max_value = 0  # Initialize max_value to find the highest count
        matrices = []  # Store matrices to later use in plots

        # Extract matrices and find the maximum value
        for name, dataset in data.items():
            matrix = self.extract_O_overlap_matrix(self.corpora[dataset]['splits'][split])
            matrices.append((name, matrix))
            max_matrix_value = matrix.to_numpy().max()
            if max_matrix_value > max_value:
                max_value = max_matrix_value

        col = 1
        for name, matrix in matrices:
            mask_lower = np.tril(np.ones_like(matrix, dtype=bool))
            lower_triangle = matrix.mask(~mask_lower)
            text_data = np.where(lower_triangle.isnull(), '', lower_triangle.fillna(0).astype(int).astype(str))

            fig.add_trace(
                go.Heatmap(
                    z=lower_triangle,
                    x=lower_triangle.columns,
                    y=lower_triangle.index,
                    coloraxis="coloraxis",
                    text=text_data,
                    texttemplate="%{text}",
                    hoverinfo="text+z"
                ),
                row=1, col=col
            )
            col += 1

        split_title = ' for the Training Split' if split == 'train' else ' for the Testing Split'
        fig.update_layout(
            title_text='Word Type Overlap Across Entity Tags' + split_title,
            template="plotly_white",
            height=600,
            width=1200,
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            xaxis=dict(showgrid=False),
            xaxis2=dict(showgrid=False),
            yaxis=dict(showgrid=False),
            yaxis2=dict(showgrid=False)
        )
        fig.show()
        
    def visualize_matrices_one_scale(self, dataset):
        """
        Visualize word type overlap matrices for each split of a single dataset.
        Displays one heatmap per split side-by-side with shared color scale.
        
        Parameters:
            dataset (dict): A dataset containing 'splits' with 'train', 'dev', 'test' keys.
        """
        splits = dataset['splits']
        fig = make_subplots(
            rows=1, cols=len(splits),
            subplot_titles=[split.capitalize() for split in splits],
            horizontal_spacing=0.1
        )

        max_value = 0
        matrices = []

        # Calculate matrices and find global max for color scaling
        for split_name, split_data in splits.items():
            matrix = self.extract_O_overlap_matrix(split_data)
            matrices.append((split_name, matrix))
            max_matrix_value = matrix.to_numpy().max()
            if max_matrix_value > max_value:
                max_value = max_matrix_value

        # Plot each heatmap
        for col, (split_name, matrix) in enumerate(matrices, start=1):
            mask_lower = np.tril(np.ones_like(matrix, dtype=bool))
            lower_triangle = matrix.mask(~mask_lower)
            text_data = np.where(lower_triangle.isnull(), '', lower_triangle.fillna(0).astype(int).astype(str))

            fig.add_trace(
                go.Heatmap(
                    z=lower_triangle,
                    x=lower_triangle.columns,
                    y=lower_triangle.index,
                    colorscale="RdBu_r",
                    zmin=0,
                    zmax=max_value,
                    colorbar=dict(title="Counts") if col == len(matrices) else None,
                    showscale=(col == len(matrices)),  # only show colorbar on the last subplot
                    text=text_data,
                    texttemplate="%{text}",
                    hoverinfo="text+z"
                ),
                row=1, col=col
            )


        fig.update_layout(
            title_text='Word Type Overlap Across Entity Tags (Each Dataset Split)',
            template="plotly_white",
            height=600,
            width=1200,
            coloraxis=dict(colorscale='RdBu_r', cmin=0, cmax=max_value, colorbar=dict(title="Counts")),
            xaxis=dict(showgrid=False),
            xaxis2=dict(showgrid=False),
            yaxis=dict(showgrid=False),
            yaxis2=dict(showgrid=False)
        )
        fig.show()


In [19]:

# Usage example
tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC', 'O']
ANERCorp = corpora['ANERCorp_CamelLab']
overlap_matrix = OverlapMatrixOneScale(ANERCorp, tags)
# overlap_matrix.visualize_matrices({'ANERCorp': 'ANERCorp_CamelLab', 'CoNLL-2003': 'conll2003'}, 'train')
overlap_matrix.visualize_matrices_one_scale(ANERCorp)


In [20]:
visualizer = EntityTagTTRVisualizer(df)
visualizer.preprocess_data()
visualizer.plot_faceted_bar_chart(
    metric='TWR',
    text='TWR',
    title='Type-to-Word Ratio (TWR) Across Entity Tags in ANERCorp (Train and Test Splits)'
)

In [21]:
class EntityTagDistributionAnalyzer:
    def __init__(self, dataset, tags_of_interest):
        """
        Initialize the analyzer with a single dataset and tags of interest.

        Parameters:
            dataset (dict): The dataset containing splits.
            tags_of_interest (list): List of tags to analyze.
        """
        self.dataset = dataset  # Single dataset only
        self.tags_of_interest = tags_of_interest

    def calculate_distributions(self, splits):
        all_data = []

        for split in splits:
            data = self.dataset['splits'][split]
            tag_word_counts = {tag: Counter() for tag in self.tags_of_interest}

            for tag in self.tags_of_interest:
                tag_word_counts[tag].update(
                    w for sen in data for w, t in zip(sen['words'], sen['tags']) if t.replace('PERS', 'PER') == tag
                )

            for tag, counter in tag_word_counts.items():
                frequencies = np.array(list(counter.values()))
                if frequencies.size > 0:
                    mean_freq = round(np.mean(frequencies), 4)
                    median_freq = round(np.median(frequencies), 4)
                    std_dev = round(np.std(frequencies), 2)
                else:
                    mean_freq, median_freq, std_dev = 0, 0, 0

                all_data.append({
                    'Split': split,
                    'Tag': tag,
                    'Mean Frequency': mean_freq,
                    'Median Frequency': median_freq,
                    'Standard Deviation': std_dev
                })

        return pd.DataFrame(all_data)
    def visualize_distributions(self, df, metric, title):
        """
        Visualize metrics for a single dataset across splits using subplots.

        Parameters:
            df (pd.DataFrame): Data with 'Tag', 'Split', and metric columns.
            metric (str): Metric to visualize (e.g., 'Mean Frequency').
            title (str): Plot title.
        """
        split_mapping = {'train': 'Train', 'test': 'Test', 'dev': 'Dev'}
        df['Split'] = df['Split'].replace(split_mapping)

        fig = px.bar(
            df,
            x='Tag',
            y=metric,
            # color='Tag',
            text=metric,
            barmode='group',
            facet_row='Split',
            title=title
        )
        fig.update_layout(
            template="plotly_white",
            height=600,
            width=400 * df['Split'].nunique(),
            xaxis_title='Entity Tag',
            yaxis_title=metric,
            showlegend=False
        )
        fig.show()



In [22]:
tags_of_interest = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
ANERCorp = corpora['ANERCorp_CamelLab']

analyzer = EntityTagDistributionAnalyzer(ANERCorp, tags_of_interest)
distributions = analyzer.calculate_distributions(['train', 'test'])

analyzer.visualize_distributions(
    distributions,
    metric='Standard Deviation',
    title='Std Dev of Word Frequencies per Entity Tag Across Dataset Splits'
)


In [23]:
pd.DataFrame(Counter([w for sen in corpora['ANERCorp_CamelLab']['splits']['train'] for w,t in zip(sen['words'], sen['tags']) if t =='I-ORG']).items()).sort_values(by=1, ascending=False).head(50)

,0,1
7,المتحدة,178
6,الله,93
2,الأمن,49
29,العربية,21
57,الدولي,20
47,الامن,19
96,يونايتد,18
400,التجاري,17
3,الدولية,15
28,الدول,13


# OOV

In [24]:
class OOVAnalyzer:
    def __init__(self, dataset, tags_of_interest):
        self.dataset = dataset
        self.tags_of_interest = tags_of_interest

    def calculate_oov_rates(self):
        """
        Computes OOV rates for each entity tag across all splits (vs train).
        """
        results = []

        # Train vocabulary per tag
        train_words = {tag: set() for tag in self.tags_of_interest}
        for sentence in self.dataset['splits']['train']:
            for word, tag in zip(sentence['words'], sentence['tags']):
                tag = tag.replace('PERS', 'PER')
                if tag in self.tags_of_interest:
                    train_words[tag].add(word)

        # Compare all other splits to train
        for split in self.dataset['splits']:
            if split == 'train':
                continue
            split_words = {tag: set() for tag in self.tags_of_interest}
            for sentence in self.dataset['splits'][split]:
                for word, tag in zip(sentence['words'], sentence['tags']):
                    tag = tag.replace('PERS', 'PER')
                    if tag in self.tags_of_interest:
                        split_words[tag].add(word)

            for tag in self.tags_of_interest:
                oov_words = split_words[tag] - train_words[tag]
                total = len(split_words[tag])
                rate = len(oov_words) / total if total else 0
                results.append({
                    'Split': split,
                    'Tag': tag,
                    'OOV Words Count': len(oov_words),
                    'Total Unique Words': total,
                    'OOV Rate': round(rate, 4)
                })

        return pd.DataFrame(results)

    def visualize_oov_rates(self, df):
        df['Split'] = df['Split'].str.capitalize()

        fig = px.bar(
            df,
            x="Tag",
            y="OOV Rate",
            color="Split",
            text="OOV Rate",
            barmode="group",
            title="OOV Rates by Tag Across Splits"
        )
        fig.update_traces(textposition="outside")
        fig.update_layout(template="plotly_white", height=500, width=1000)
        fig.show()


In [25]:
tags = ['B-LOC', 'I-LOC', 'B-PER', 'I-PER', 'B-ORG', 'I-ORG', 'B-MISC', 'I-MISC']
splits = ['train', 'test']
dataset = corpora['ANERCorp_CamelLab']

# OOV
oov = OOVAnalyzer(dataset, tags)
oov_df = oov.calculate_oov_rates()
oov.visualize_oov_rates(oov_df)


In [26]:
oov_stats_per_tag = calculate_oov_rates_per_tag(corpora)

# Convert results to a suitable format for display, such as a DataFrame
import pandas as pd

# Transform the dictionary into a DataFrame for easier viewing and analysis
data_frames = {}
for corpus, tags_data in oov_stats_per_tag.items():
    if isinstance(tags_data, dict):  # Ensure there's data to process
        df = pd.DataFrame.from_dict(tags_data, orient='index')
        data_frames[corpus] = df

# Creating a single DataFrame from the dictionary
all_data = pd.concat({
    k: v for k, v in data_frames.items()
}, axis=0)

# Rename the levels for clarity
all_data.index.names = ['Corpus', 'Tag']
all_data

OOV Words Count  Total Unique Words in Test  \
Corpus            Tag                                                   
ANERCorp_CamelLab B-LOC                98                         268   
                  B-PER               347                         506   
                  I-PER               288                         419   
                  B-ORG               131                         210   
                  I-LOC                18                          34   
                  I-ORG               106                         151   
                  B-MISC               93                         123   
                  I-MISC               93                         111   
conll2003         B-LOC               187                         475   
                  B-PER               519                         857   
                  I-PER               610                         797   
                  I-LOC                47                         102   
                  B-MISC              150                         274   
                  I-MISC               50                          94   
                  B-ORG               267                         658   
                  I-ORG               147                         379   

                          OOV Rate  
Corpus            Tag               
ANERCorp_CamelLab B-LOC     0.3657  
                  B-PER     0.6858  
                  I-PER     0.6874  
                  B-ORG     0.6238  
                  I-LOC     0.5294  
                  I-ORG     0.7020  
                  B-MISC    0.7561  
                  I-MISC    0.8378  
conll2003         B-LOC     0.3937  
                  B-PER     0.6056  
                  I-PER     0.7654  
                  I-LOC     0.4608  
                  B-MISC    0.5474  
                  I-MISC    0.5319  
                  B-ORG     0.4058  
                  I-ORG     0.3879

In [27]:
dataset = 'conll2003'
entity_tag = 'B-PER'
train = set([w for sen in corpora[dataset]['splits']['train'] for w, t in zip(sen['words'], sen['tags']) if t == entity_tag])
test = set([w for sen in corpora[dataset]['splits']['test'] for w, t in zip(sen['words'], sen['tags']) if t == entity_tag])
len(test - train)

519

# Entity

In [28]:
class EntityDistributionAnalyzer:
    def __init__(self, dataset, schemes, splits):
        """
        Initialize the Entity Distribution Analyzer for a single dataset.

        Parameters:
            dataset (dict): The dataset containing splits.
            schemes (list): Annotation schemes (e.g., ['IOB1', 'IOB2']).
            splits (list): Data splits to analyze (e.g., ['train', 'test']).
        """
        self.dataset = dataset
        self.schemes = schemes
        self.splits = splits

    def get_entity_distribution(self):
        """
        Computes the distribution of entity types across splits and schemes.

        Returns:
            pd.DataFrame: DataFrame containing entity type counts and proportions.
        """
        data = []

        for split in self.splits:
            for scheme in self.schemes:
                y_true = [sen['tags'] for sen in self.dataset['splits'][split]]

                if scheme == 'IOB1':
                    true_entities = get_entities(y_true)
                else:
                    detected_scheme = auto_detect(y_true, False)
                    parsed = Entities(y_true, detected_scheme, False)
                    true_entities = [entity.to_tuple()[1:] for sen in parsed.entities for entity in sen]

                raw_counts = Counter([etype[0].replace('PERS', 'PER') for etype in true_entities])
                total = sum(raw_counts.values())

                for entity_type, count in raw_counts.items():
                    percentage = round((count / total) * 100, 2) if total > 0 else 0
                    data.append({
                        'Split': split,
                        'Annotation Scheme': scheme,
                        'Entity Type': entity_type,
                        'Raw Count': count,
                        'Proportion': percentage
                    })

        return pd.DataFrame(data)

    def visualize_entity_distribution(self, df, metric):
        """
        Visualize entity distributions across splits and schemes.

        Parameters:
            df (pd.DataFrame): The DataFrame containing the metrics.
            metric (str): Metric to visualize, e.g., 'Proportion' or 'Raw Count'.
        """
        df['Split'] = df['Split'].str.capitalize()

        fig = px.bar(
            df,
            x='Entity Type',
            y=metric,
            # color='Split',
            text='Raw Count',
            barmode='group',
            facet_col='Annotation Scheme',
            facet_row='Split',
            category_orders={"Annotation Scheme": self.schemes},
            labels={"Proportion": "Proportion (%)"},
            title=f"{metric} of Entity Types Across Annotation Schemes and Splits"
        )

        fig.update_layout(
            template="plotly_white",
            height=700,
            width=1200,
            plot_bgcolor='rgba(0,0,0,0)'
        )

        fig.show()


In [29]:
schemes = ['IOB1', 'IOB2']
splits = ['train', 'test']
dataset = corpora['ANERCorp_CamelLab']  # Single dataset

analyzer = EntityDistributionAnalyzer(dataset, schemes, splits)
entity_distribution_df = analyzer.get_entity_distribution()
analyzer.visualize_entity_distribution(entity_distribution_df, 'Proportion')


In [30]:

class EntityLengthAnalyzer:
    def __init__(self, dataset, schemes, splits):
        self.dataset = dataset
        self.schemes = schemes
        self.splits = splits

    
    def calculate_entity_lengths(self, entities, strict=False):
        from collections import defaultdict
        import numpy as np

        entity_lengths = defaultdict(list)
        for etype, start, end in entities:
            length = end - start if strict else end - start + 1
            entity_lengths[etype.replace('PERS', 'PER')].append(length)

        entity_stats = {}  # <-- renamed to avoid conflict with scipy.stats

        for tag, lengths in entity_lengths.items():
            lengths = np.array(lengths)
            mode_result = stats.mode(lengths, keepdims=True)
            entity_stats[tag] = {
                'Mean Length': round(np.mean(lengths), 2),
                'Median Length': round(np.median(lengths), 2),
                'Standard Deviation': round(np.std(lengths), 2),
                'Interquartile Range (IQR)': round(stats.iqr(lengths), 2),
                'Mode Length': int(mode_result.mode[0]) if len(mode_result.mode) > 0 else None,
                'Min Length': int(np.min(lengths)),
                'Max Length': int(np.max(lengths))
            }

        return entity_stats

    def process(self):
        results = []

        for split in self.splits:
            for scheme_name in self.schemes:
                tags = [s['tags'] for s in self.dataset['splits'][split]]
                strict = (scheme_name == 'IOB2')

                if strict:
                    scheme = auto_detect(tags, False)
                    entities = Entities(tags, scheme, False)
                    entity_list = [e.to_tuple()[1:] for s in entities.entities for e in s]
                else:
                    entity_list = get_entities(tags)

                stats = self.calculate_entity_lengths(entity_list, strict)
                for tag, metrics in stats.items():
                    metrics.update({'Entity Type': tag, 'Scheme': scheme_name, 'Split': split})
                    results.append(metrics)

        return pd.DataFrame(results)

    def visualize(self, df, metric):
        df['Split'] = df['Split'].str.capitalize()

        fig = px.bar(
            df,
            x='Entity Type',
            y=metric,
            color='Split',
            text=metric,
            barmode='group',
            facet_col='Scheme',
            title=f"{metric} by Entity Type, Scheme, and Split"
        )
        fig.update_layout(template="plotly_white", height=700, width=1200)
        fig.show()


In [31]:
# Parameters
# schemes = ['IOB1', 'IOB2']
schemes = ['IOB2']
splits = ['train', 'test']  # Include dev if present
dataset = corpora['ANERCorp_CamelLab']  # Single dataset

# Initialize once
length_analyzer = EntityLengthAnalyzer(dataset, schemes, splits)

# Compute once
entity_length_df = length_analyzer.process()

# Visualize multiple metrics
for metric in ["Mean Length", "Median Length", "Standard Deviation", "Interquartile Range (IQR)", "Max Length", "Min Length"]:
    length_analyzer.visualize(entity_length_df, metric)
